In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [24]:
url = "https://www.espncricinfo.com/records/tournament/team-match-results/indian-premier-league-2023-15129"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

page = requests.get(url=url, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [33]:
matches_list = []
matches = soup.select("table > tbody > tr")

for match in matches:
    m = {
        "team_1": match.select_one("td:nth-child(1)").text,
        "team_2": match.select_one("td:nth-child(2)").text,
        "winner": match.select_one("td:nth-child(3)").text,
        "margin": match.select_one("td:nth-child(4)").text,
    }
    matches_list.append(m)

In [34]:
df = pd.DataFrame(matches_list)
df

,team_1,team_2,winner,margin
0,Titans,Super Kings,Titans,5 wickets
1,Punjab Kings,KKR,Punjab Kings,7 runs
2,Super Giants,Capitals,Super Giants,50 runs
3,Sunrisers,Royals,Royals,72 runs
4,RCB,Mumbai,RCB,8 wickets
...,...,...,...,...
69,RCB,Titans,Titans,6 wickets
70,Super Kings,Titans,Super Kings,15 runs
71,Super Giants,Mumbai,Mumbai,81 runs
72,Titans,Mumbai,Titans,62 runs


In [35]:
names = {
    "Titans": "GT",
    "Punjab Kings": "PBKS",
    "Super Giants": "LSG",
    "Sunrisers": "SRH",
    "Royals": "RR",
    "Capitals": "DC",
    "Mumbai": "MI",
    "Super Kings": "CSK",
    "RCB": "RCB",
    "KKR": "KKR",
    "no result": "no result"
    
}

df["team_1"] = df["team_1"].map(names)
df["team_2"] = df["team_2"].map(names)
df["winner"] = df["winner"].map(names)

print(df.to_string())

   team_1 team_2     winner     margin
0      GT    CSK         GT  5 wickets
1    PBKS    KKR       PBKS     7 runs
2     LSG     DC        LSG    50 runs
3     SRH     RR         RR    72 runs
4     RCB     MI        RCB  8 wickets
5     CSK    LSG        CSK    12 runs
6      DC     GT         GT  6 wickets
7      RR   PBKS       PBKS     5 runs
8     KKR    RCB        KKR    81 runs
9     LSG    SRH        LSG  5 wickets
10     RR     DC         RR    57 runs
11     MI    CSK        CSK  7 wickets
12     GT    KKR        KKR  3 wickets
13    SRH   PBKS        SRH  8 wickets
14    RCB    LSG        LSG   1 wicket
15     DC     MI         MI  6 wickets
16    CSK     RR         RR     3 runs
17   PBKS     GT         GT  6 wickets
18    KKR    SRH        SRH    23 runs
19    RCB     DC        RCB    23 runs
20    LSG   PBKS       PBKS  2 wickets
21     MI    KKR         MI  5 wickets
22     GT     RR         RR  3 wickets
23    RCB    CSK        CSK     8 runs
24    SRH     MI         

In [69]:
# Getting count of team wins and team played in the form of dictionary
def get_data_count(data):
    obj = {}
    for d in df[data]:
        if d in obj:
            obj[d] += 1
        else:
            obj[d] = 1
    return obj

# Getting total team wins, loses and played
def get_team_stats(team_played, team_wins):
    teams_lost = {}
    team_stats = {}
    chase = {}
    defend = {}
    
    # getting total no of matches played
    for team, count in get_data_count("team_2").items():
        if team in team_played:
            team_played[team] += count
    
    # getting total no of matches losed
    for team in team_played.keys():
        teams_lost[team] = team_played[team] - team_wins[team]
        
    # getting chase and defend count   
    for i in range(len(df)):
        if "wickets" in df.loc[i]["margin"]:
            if df.loc[i]["winner"] in chase:
                chase[df.loc[i]["winner"]] += 1
            else:
                chase[df.loc[i]["winner"]] = 1
        else:
            if df.loc[i]["winner"] in defend:
                defend[df.loc[i]["winner"]] += 1
            else:
                defend[df.loc[i]["winner"]] = 1
        

    # combining wins, loses and played into one dictionary
    team_stats["played"] = team_played
    team_stats["wins"] = team_wins
    team_stats["loses"] = teams_lost
    team_stats["chased"] = chase
    team_stats["defended"] = defend
    team_stats["logo"] = images
    
    return team_stats
    
    
# stroing data into below variable    
team_stats = get_team_stats(get_data_count("team_1"), get_data_count("winner"))

In [65]:
# Converting dictionary into data frame
matches_df = pd.DataFrame.from_dict(team_stats)
matches_df = matches_df.drop(matches_df.index[-1])
# Sorting data frame by wins
matches_df

,played,wins,loses,chased,defended,logo
GT,17.0,11,6.0,6.0,5,"https://img1.hscicdn.com/image/upload/f_auto,t..."
PBKS,14.0,6,8.0,2.0,4,"https://img1.hscicdn.com/image/upload/f_auto,t..."
LSG,15.0,8,7.0,2.0,6,"https://img1.hscicdn.com/image/upload/f_auto,t..."
SRH,14.0,4,10.0,2.0,2,"https://img1.hscicdn.com/image/upload/f_auto,t..."
RCB,14.0,7,7.0,2.0,5,"https://img1.hscicdn.com/image/upload/f_auto,t..."
CSK,16.0,10,6.0,4.0,6,"https://img1.hscicdn.com/image/upload/f_auto,t..."
DC,14.0,5,9.0,2.0,3,"https://img1.hscicdn.com/image/upload/f_auto,t..."
RR,14.0,7,7.0,3.0,4,"https://img1.hscicdn.com/image/upload/f_auto,t..."
KKR,14.0,6,8.0,3.0,3,"https://img1.hscicdn.com/image/upload/f_auto,t..."
MI,16.0,9,7.0,6.0,3,"https://img1.hscicdn.com/image/upload/f_auto,t..."


In [66]:
matches_df["played"] = matches_df["played"].astype(int)
matches_df["loses"] = matches_df["loses"].astype(int)
matches_df["chased"] = matches_df["chased"].astype(int)
matches_df

,played,wins,loses,chased,defended,logo
GT,17,11,6,6,5,"https://img1.hscicdn.com/image/upload/f_auto,t..."
PBKS,14,6,8,2,4,"https://img1.hscicdn.com/image/upload/f_auto,t..."
LSG,15,8,7,2,6,"https://img1.hscicdn.com/image/upload/f_auto,t..."
SRH,14,4,10,2,2,"https://img1.hscicdn.com/image/upload/f_auto,t..."
RCB,14,7,7,2,5,"https://img1.hscicdn.com/image/upload/f_auto,t..."
CSK,16,10,6,4,6,"https://img1.hscicdn.com/image/upload/f_auto,t..."
DC,14,5,9,2,3,"https://img1.hscicdn.com/image/upload/f_auto,t..."
RR,14,7,7,3,4,"https://img1.hscicdn.com/image/upload/f_auto,t..."
KKR,14,6,8,3,3,"https://img1.hscicdn.com/image/upload/f_auto,t..."
MI,16,9,7,6,3,"https://img1.hscicdn.com/image/upload/f_auto,t..."


In [77]:
matches_df.to_csv("matches.csv")

In [72]:
matches_df.head()

,played,wins,loses,chased,defended,logo
GT,17,11,6,6,5,"https://img1.hscicdn.com/image/upload/f_auto,t..."
PBKS,14,6,8,2,4,"https://img1.hscicdn.com/image/upload/f_auto,t..."
LSG,15,8,7,2,6,"https://img1.hscicdn.com/image/upload/f_auto,t..."
SRH,14,4,10,2,2,"https://img1.hscicdn.com/image/upload/f_auto,t..."
RCB,14,7,7,2,5,"https://img1.hscicdn.com/image/upload/f_auto,t..."


In [80]:
batters_df = pd.read_csv("batters.csv")
bowlers_df = pd.read_csv("bowlers.csv")
matches_df = pd.read_csv("matches.csv")

with pd.ExcelWriter('ipl_2023_data.xlsx') as writer:
    matches_df.to_excel(writer, index=False, sheet_name="Matches")
    batters_df.to_excel(writer, index=False, sheet_name="Batters")
    bowlers_df.to_excel(writer, index=False, sheet_name="Bowlers")